In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from random import randrange

In [2]:
def generate_fake_data(number_of_patients=365):
  X_context = []
  X_time_series = []
  y = []  
  for _ in range(0, number_of_patients):
    age = np.random.randint(0, 101)
    sex = np.random.randint(0, 2) 
    bmi = np.random.randint(30, 40) 
    systolic_bp = np.random.randint(80, 120, size=14) 
    diastolic_bp = np.random.randint(70, 80, size=14) 
    future_systolic_bp = np.random.randint(80, 120, size=7) 
    future_diastolic_bp = np.random.randint(70, 80, size=7)
    result = np.concatenate((future_systolic_bp, future_diastolic_bp), axis=0)
    X_context.append([age, sex, bmi])
    X_time_series.append([systolic_bp,diastolic_bp])
    y.append([result])
  return np.array(X_context,dtype=float),np.array(X_time_series,dtype=float),np.array(y,dtype=float)


In [3]:
X_context,X_time_series,y = generate_fake_data()

In [4]:
X_context.shape

(365, 3)

In [5]:
X_time_series = X_time_series.reshape(365,14,2)

In [6]:
y = y.reshape(365,14,1)

In [7]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense,Dropout,Multiply,Input,BatchNormalization,Activation,LSTM
from tensorflow.keras.callbacks import EarlyStopping

In [8]:
input_context_layer = Input((3,))
context_layer = Dense(3,activation="leaky_relu")(input_context_layer)
batch_norm_layer = BatchNormalization()(context_layer)
sigmoid_layer = Activation(activation="sigmoid")(batch_norm_layer)
input_time_series_layer = Input((14,2,))
bp_sequence_layer = LSTM(3, activation='tanh')(input_time_series_layer)
multiply_layer = Multiply()([sigmoid_layer,bp_sequence_layer])
final_layer = Dense(14)(multiply_layer)
model = Model(inputs=[input_context_layer,input_time_series_layer],outputs=final_layer)

In [9]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 3)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 3)                 │              12 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization           │ (None, 3)                 │              12 │ dense[0][0]                │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_1 (InputLayer)    │ (None, 14, 2)             │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation (Activation)       │ (None, 3)                 │               0 │ batch_normalization[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm (LSTM)                   │ (None, 3)                 │              72 │ input_layer_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ multiply (Multiply)           │ (None, 3)                 │               0 │ activation[0][0],          │
│                               │                           │                 │ lstm[0][0]                 │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_1 (Dense)               │ (None, 14)                │              56 │ multiply[0][0]             │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 152 (608.00 B)

 Trainable params: 146 (584.00 B)

 Non-trainable params: 6 (24.00 B)

In [ ]:
model.compile(optimizer="adam",loss="mse")
early_stop = EarlyStopping(monitor='val_loss',patience=5)
model.fit(x=[X_context,X_time_series],y=y,batch_size=32,epochs=1000,validation_split=0.2,callbacks=[early_stop])

Epoch 1/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - loss: 7872.0933 - val_loss: 7774.7656
Epoch 2/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 7882.8848 - val_loss: 7770.7065
Epoch 3/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 7883.2783 - val_loss: 7766.8477
Epoch 4/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 7877.1748 - val_loss: 7763.1680
Epoch 5/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 7892.2886 - val_loss: 7759.4136
Epoch 6/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 7870.0908 - val_loss: 7754.4956
Epoch 7/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 7845.7300 - val_loss: 7749.9365
Epoch 8/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 7845.8491 - val_loss: 7747.6807
Epoch 9/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 7848.7539 - val_loss: 7745.8774
Epoch 10/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 7880.4863 - val_loss: 7744.1270
Epoch 11/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 7828.8574 - val_l

In [ ]:
X_context[0]

In [ ]:
X_time_series[0]

In [ ]:
losses = pd.DataFrame(model.history.history)

In [ ]:
losses[['loss','val_loss']].plot()

In [ ]:
model.save("bp.keras")